In [1]:
LOAD = True

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVC
import os
from scipy.io import wavfile
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
import pickle
from tqdm import tqdm
from sklearn.externals import joblib

In [3]:
def preprocess(data,input_length):
    if len(data) > input_length:
                max_offset = len(data) - input_length
                offset = np.random.randint(1,max_offset)
                data = data[offset:(input_length+offset)]
    else:
        if input_length > len(data):
            max_offset = input_length - len(data)
            offset = np.random.randint(1,max_offset)
        else:
            offset = 0
        data = np.pad(data, (offset, input_length - len(data) - offset), "constant")
    max_data = np.max(data)
    min_data = np.min(data)
    data = ((data-min_data)/(max_data-min_data+1e-6)) - 0.5
    return data

In [4]:
train = pd.read_csv("../data/train.csv")
train.head()

,fname,label,manually_verified
0,00044347.wav,Hi-hat,0
1,001ca53d.wav,Saxophone,1
2,002d256b.wav,Trumpet,0
3,0033e230.wav,Glockenspiel,1
4,00353774.wav,Cello,1


In [9]:
location = '../data/audio_train'
m=[]
count=0
rate =  44100
files = np.array(os.listdir(location))
trainX = []
trainY = []
for clip in tqdm(files):
    rate, data = wavfile.read('../data/audio_train/' + clip)
    record = train[train.fname == clip]
    label = record['label'].values[0]
    trainY.append(label)
    data = preprocess(data, rate*2)
    trainX.append(data)
    count+=1

  0%|          | 0/9473 [00:00<?, ?it/s]/home/ankur16225/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in short_scalars
  from ipykernel import kernelapp as app
100%|██████████| 9473/9473 [02:29<00:00, 63.41it/s]


In [ ]:
location = '../data/audio_test'
m=[]
rate =  44100
files = np.array(os.listdir(location))
testX = []
testY = []
for clip in tqdm(files):
    rate, data = wavfile.read('../data/audio_test/' + clip)
    record = test[test.fname == clip]
    label = record['label'].values[0]
    if label!=None:
        testY.append(label)
        data = preprocess(data, rate*2)
        testX.append(data)

In [10]:
trainX = np.array(trainX)

(9473, 88200)


In [11]:
lookupTable1, trainY = np.unique(trainY, return_inverse=True)
print(lookupTable1[:5], trainY[:5])
print(len(lookupTable1), len(trainY))

['Acoustic_guitar' 'Applause' 'Bark' 'Bass_drum' 'Burping_or_eructation'] [40 20  4  8 23]
41 9473


In [6]:
if not LOAD:
    svc = SVC()
    svc.fit(trainX, trainY) 
    # Save to file in the current working directory
    joblib_file = "joblib_model.pkl"  
    joblib.dump(svc, joblib_file)

In [ ]:
if LOAD:
    # load model
#     svc = joblib.load('joblib_model.pkl')

In [ ]:
trainScore = svc.score(trainX, trainY)

In [25]:
print(trainScore)

0.2892431120025335


In [8]:
testScore = svc.score(testX, testY)
print(testScore)

0.1373245213586
